Author
- Jovan Karuna Cahyadi (13518024)
- Ricky Fernando (13518062)
- Stefanus Stanley Yoga Setiawan (13518122)
- William (13518138)

# External Library


In [ ]:
import pandas as pd
import math
import numpy as np
import json
from json import JSONEncoder
from pprint import pprint

# Utility Function

In [ ]:
#######################
# Activation Function #
#######################

def relu(x):
  return np.maximum(0, x)

def linear(x):
  return x

def sigmoid(x):
  return 1/(1+np.exp(-x))

def softmax(x):
  return np.exp(x) / np.sum(np.exp(x), axis=0)

####################
# Scoring Function #
####################

def accuracy_score(y_pred, y_test):
  count = 0
  total = len(y_pred)
  for i in range(total):
    if(y_pred[i] != y_test[i]):
      count += 1
  return 1 - count/total

from keras import backend as K
def recall_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

#######################
# External File Utils #
#######################
def export_to_json(data, filename):
  
  class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

  with open(filename, "w") as json_file:  
    json.dump(data, json_file, cls=NumpyArrayEncoder) 
  

def import_from_json(filename):
  with open(filename, 'r') as json_file: 
    data = json.load(json_file)
  if data is not None : 
    return data
  else :
    print("No data found!")

def activation_func_identifier(func_name):
  return {
    "sigmoid" : sigmoid,
    "softmax" : softmax,
    "linear" : linear,
    "relu" : relu
  }.get(func_name)

def activation_func_dump(func):
  return {
    sigmoid : "sigmoid",
    softmax : "softmax",
    linear : "linear",
    relu : "relu"
  }.get(func)

def create_history(ann):
  for i, hist in enumerate(ann.history):
    if i == 0:
      print('Input Layer')
    else:
      print(f'Layer {i}')
    for j, value in enumerate(hist):
      if i == 0:
        print(f'    Features {j} = {value}')
      else:
        print(f'    σ(Out Nodes {j}) = {value}')

# Artificial Neural Network Representation

In [ ]:
class HiddenLayer():
  """
  Attributes:
    weights (np.array) -> with shape (n_nodes, n_features)
      weights[nodes][features]
    prev_res (np.array) -> with shape (n_features, n_data)
      prev_res[features][n]
    out (np.array) -> with shape (n_nodes, n_data)
    bias (np.array) -> with shape (n_nodes)
    activation_function (function) -> activation function for this layer
  
  Method:
    calculate_out -> dot product between input and nodes weight
  """
  def __init__(self, nodes, activation_function, prev_res=None):
    """
    Parameters:
      nodes (tuple(int, int)) -> Create matrix weights
      activation_function (function) -> activation function for this layer
      prev_res (np.array) -> with shape (n_nodes_before, n_data), first layer
      shape is (n_features, n_data), next layer will be initialize with None
      because no output yet from nodes before
    """
    self.weights = np.random.uniform(0, 0, nodes)
    self.out = None
    self.prev_res = prev_res
    self.bias = [0 for i in range(self.weights.shape[0])]
    self.activation_function = activation_function

  def calculate_out(self, prev_res):
    """
    Calculate dot product between weights and inputs
    Formula:
      activation(Weights . value_before + bias)

    Parameters:
      prev_res (np.array) -> Input given to layer
    """
    self.prev_res = prev_res
    mult = self.weights @ self.prev_res
    for i in range(self.weights.shape[0]):
      mult[i] += self.bias[i]
    self.out = self.activation_function(mult)

  def __str__(self):
    return f'{self.weights}'

In [ ]:
class ANNClassifier():
  """
  Attributes:
    x (np.array) -> with shape (n_features, n_data)
    y (np.array) -> with shape (n_data, 1)
    learning_rate (float) -> learning_rate for training
    metrics (function) -> metrics for evaluation
    verbose (int) -> output message training every epoch
    layers (list(HiddenLayer)) -> last layer is output layer
    loss (list(float)) -> loss history 
    score (list(float)) -> score history
    history (list(float)) -> input from every layer
    
  Method:
    __feed_forward -> feed forward in ANN
    __back_propagation -> back propagation in ANN
    add -> insert layer to ANN
    train -> train model
    predict -> predict data
    save -> saving model
    load -> loading model
    compile -> compile model
  """
  def __init__(self, x, y, learning_rate, metrics=accuracy_score, verbose=1):
    self.x = x.T
    self.y = y
    self.learning_rate = learning_rate
    self.metrics = metrics
    self.verbose = verbose
    self.layers = []
    self.loss = []
    self.score = []
    self.history = []

  def __feed_forward(self):
    """
    Feed Forward method used in ANN which calculate input and produce output
    using dot product in every layer

    Formula:
      res[0] = input
      res[1] = activation(Weights[0] . res[0] + bias[0])
      res[2] = activation(Weights[1] . res[1] + bias[1])
      .
      ..
      ...
      res[n] = activation(Weights[n-1] . res[n-1] + bias[n-1])
      output = activation(Weights[n] . res[n] ]+ bias[n])
    """
    prev_res = self.layers[0].prev_res
    for layer in self.layers:
      layer.calculate_out(prev_res)
      prev_res = layer.out
  
  def __back_propagation(self):
    pass

  def train(self):
    pass

  def add(self, nodes, activation_function=sigmoid):
    """
    Add another Hidden Layer into ANNClassifier
    
    Parameters:
      nodes (int) -> How many nodes in the layer
      activation_function (function) -> What activation function used in the 
      layer
    """
    if len(self.layers) == 0:
        weights_shape = (nodes, self.x.shape[0])
        prev_res = self.x
        layer = HiddenLayer(weights_shape, activation_function, prev_res)
    else:
        weights_shape = (nodes, self.layers[-1].weights.shape[0])
        layer = HiddenLayer(weights_shape, 
                            activation_function)
    self.layers.append(layer)

  def predict(self, in_val):
    """
    Predict given data

    Parameters:
      in_val (np.array) -> with shape (n_data, n_features)
    
    Return:
      prev_res -> predicted value
    """
    self.history = []
    prev_res = np.array(in_val).T
    for layer in self.layers:
      self.history.append(prev_res)
      layer.calculate_out(prev_res)
      prev_res = layer.out
    self.history.append(prev_res)
    return prev_res.T
  
  def save(self, filename):
    """
    Save layers, layers weights and activation_function
    """
    layers = list(map(
      lambda layer : {
        "nodes" : layer.weights.shape[0],
        "weights" : layer.weights,
        "bias" : layer.bias,
        "activation" : activation_func_dump(layer.activation_function)
      }, 
      self.layers
    ))

    if (self.verbose > 1):
      print("Export:")
      pprint(layers)

    export_to_json(layers, filename)


  def load(self, filename):
    """
    Load layers, layers weights and activation function
    """
    layers = import_from_json(filename)

    if (self.verbose > 1):
      print("Import : ")
      pprint(layers)

    for count, layer in enumerate(layers) : 
      self.add(layer['nodes'], activation_function = 
                activation_func_identifier(layer['activation']))
      self.layers[-1].weights = np.array(layer['weights'])
      self.layers[-1].bias = layer['bias']
  
  def summary(self):
    """
    Summary of the model (layers, shape, params, and weight)
    """
    summary = []
    dim = self.x.shape[0]
    for count, layer in enumerate(self.layers) :
      param = (dim + 1) * layer.weights.shape[0] 
      dim = layer.weights.shape[0]
      summary.append({
          'layer': f'dense_{count} (Dense)',
          'output': layer.weights.shape[0],
          'activation': layer.activation_function.__name__,
          'param': param,
          'weight': layer.weights
      })

    print("SUMMARY")
    print("====================================")
    print(f'Layer (Type)\t: Input Layer')
    print(f'Activation\t: None')
    print(f'Output Shape\t: {self.x.shape[0]}')
    print(f'Param\t\t: -')

    total_param = 0
    for layer in summary:
      total_param += layer['param']
      print("====================================")
      print(f'Layer (Type)\t: {layer["layer"]}')
      print(f'Activation\t: {layer["activation"]}')
      print(f'Output Shape\t: {layer["output"]}')
      print(f'Param\t\t: {layer["param"]}')
      print("Weight\t\t:")
      for count, weight in enumerate(layer["weight"]):
        print(f'\tnode_{count} {weight}')
        
    print("====================================")
    print(f"Total params : {total_param}")

# Testing

## XoR

In [ ]:
x = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]])
y = np.array([0, 1, 1, 0])
y = y.reshape(-1, 1)

## Sigmoid

In [ ]:
ann = ANNClassifier(x, y, learning_rate=0.1, verbose=100)
ann.add(2, activation_function=sigmoid)
ann.add(1, activation_function=sigmoid)

In [ ]:
ann.layers[0].weights = np.array([[20, 20],
                                  [-20, -20]])

ann.layers[0].bias = np.array([-10, 30])

ann.layers[1].weights = np.array([[20, 20]])

ann.layers[1].bias = [-30]

In [ ]:
y_pred = ann.predict(x)
y_pred

array([[4.54391049e-05],
       [9.99954520e-01],
       [9.99954520e-01],
       [4.54391049e-05]])

In [ ]:
ann.summary()

SUMMARY
Layer (Type)	: Input Layer
Activation	: None
Output Shape	: 2
Param		: -
Layer (Type)	: dense_0 (Dense)
Activation	: sigmoid
Output Shape	: 2
Param		: 6
Weight		:
	node_0 [20 20]
	node_1 [-20 -20]
Layer (Type)	: dense_1 (Dense)
Activation	: sigmoid
Output Shape	: 1
Param		: 3
Weight		:
	node_0 [20 20]
Total params : 9


In [ ]:
create_history(ann)

Input Layer
    Features 0 = [0 0 1 1]
    Features 1 = [0 1 0 1]
Layer 1
    σ(Out Nodes 0) = [4.53978687e-05 9.99954602e-01 9.99954602e-01 1.00000000e+00]
    σ(Out Nodes 1) = [1.00000000e+00 9.99954602e-01 9.99954602e-01 4.53978687e-05]
Layer 2
    σ(Out Nodes 0) = [4.54391049e-05 9.99954520e-01 9.99954520e-01 4.54391049e-05]


## Relu & Linear

In [ ]:
ann = ANNClassifier(x, y, learning_rate=0.1, verbose=100)
ann.add(2, activation_function=relu)
ann.add(1, activation_function=linear)

In [ ]:
ann.layers[0].weights = np.array([[1, 1],
                                  [1, 1]])

ann.layers[0].bias = np.array([0, -1])

ann.layers[1].weights = np.array([[1, -2]])

ann.layers[1].bias = [0]

In [ ]:
y_pred = ann.predict(x)
y_pred

array([[0],
       [1],
       [1],
       [0]])

In [ ]:
ann.summary()

SUMMARY
Layer (Type)	: Input Layer
Activation	: None
Output Shape	: 2
Param		: -
Layer (Type)	: dense_0 (Dense)
Activation	: relu
Output Shape	: 2
Param		: 6
Weight		:
	node_0 [1 1]
	node_1 [1 1]
Layer (Type)	: dense_1 (Dense)
Activation	: linear
Output Shape	: 1
Param		: 3
Weight		:
	node_0 [ 1 -2]
Total params : 9


In [ ]:
create_history(ann)

Input Layer
    Features 0 = [0 0 1 1]
    Features 1 = [0 1 0 1]
Layer 1
    σ(Out Nodes 0) = [0 1 1 2]
    σ(Out Nodes 1) = [0 0 0 1]
Layer 2
    σ(Out Nodes 0) = [0 1 1 0]


## Softmax

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

In [ ]:
y_softmax = np.array([[1, 0],
                      [0, 1],
                      [0, 1],
                      [1, 0]])

In [ ]:
model = Sequential()
model.add(Dense(8, input_dim=2, activation="sigmoid"))
model.add(Dense(2, activation="softmax"))

sgd = SGD(lr=0.1)

model.compile(loss='categorical_crossentropy', 
              optimizer=sgd, 
              metrics=['accuracy'])
model.fit(x, y_softmax, epochs=1000, batch_size=1, verbose=0)
model.predict(x)

array([[0.9121973 , 0.08780274],
       [0.14021187, 0.8597881 ],
       [0.11261296, 0.88738704],
       [0.80164397, 0.198356  ]], dtype=float32)

In [ ]:
# Layer 1
weights_layer1 = model.layers[0].get_weights()[0]
biases_layer1 = model.layers[0].get_weights()[1]
# Layer 2
weights_layer2 = model.layers[1].get_weights()[0]
biases_layer2 = model.layers[1].get_weights()[1]

In [ ]:
ann = ANNClassifier(x, y_softmax, learning_rate=0.1, verbose=100)
ann.add(8, activation_function=sigmoid)
ann.add(2, activation_function=softmax)

In [ ]:
ann.layers[0].weights = weights_layer1.T

ann.layers[0].bias = biases_layer1

ann.layers[1].weights = weights_layer2.T

ann.layers[1].bias = biases_layer2

In [ ]:
y_pred = ann.predict(x)
y_pred

array([[0.91219728, 0.08780272],
       [0.14021183, 0.85978817],
       [0.11261296, 0.88738704],
       [0.80164388, 0.19835612]])

In [ ]:
ann.summary()

SUMMARY
Layer (Type)	: Input Layer
Activation	: None
Output Shape	: 2
Param		: -
Layer (Type)	: dense_0 (Dense)
Activation	: sigmoid
Output Shape	: 8
Param		: 24
Weight		:
	node_0 [ 3.2450957 -4.2258496]
	node_1 [-2.5172458 -2.9276252]
	node_2 [-0.03413637 -0.6484623 ]
	node_3 [-3.300996  2.278239]
	node_4 [-0.3874242   0.06884778]
	node_5 [-1.6303849   0.60353094]
	node_6 [ 0.4656466  -0.08640727]
	node_7 [-0.645144  -0.4315381]
Layer (Type)	: dense_1 (Dense)
Activation	: softmax
Output Shape	: 2
Param		: 18
Weight		:
	node_0 [-2.7685611   2.6373317   0.31633526 -2.588205    0.5865817  -0.36277023
  0.21714655 -0.06120121]
	node_1 [ 3.1873586  -1.6696097  -0.08435065  2.2522638   0.69639504  1.2915646
 -0.78183573 -0.21638407]
Total params : 42


In [ ]:
create_history(ann)

Input Layer
    Features 0 = [0 0 1 1]
    Features 1 = [0 1 0 1]
Layer 1
    σ(Out Nodes 0) = [0.13390451 0.00225417 0.79870617 0.05480436]
    σ(Out Nodes 1) = [0.55563224 0.06272768 0.09163844 0.00537067]
    σ(Out Nodes 2) = [0.3396337  0.21192024 0.33201992 0.20627519]
    σ(Out Nodes 3) = [0.22513593 0.73928502 0.0105923  0.09459821]
    σ(Out Nodes 4) = [0.29180947 0.30623845 0.21856709 0.2305535 ]
    σ(Out Nodes 5) = [0.34805526 0.4939834  0.09466305 0.16050809]
    σ(Out Nodes 6) = [0.33142703 0.31256708 0.44124982 0.42006695]
    σ(Out Nodes 7) = [0.33576324 0.24716862 0.20959368 0.14692635]
Layer 2
    σ(Out Nodes 0) = [0.91219728 0.14021183 0.11261296 0.80164388]
    σ(Out Nodes 1) = [0.08780272 0.85978817 0.88738704 0.19835612]


## Total Params

In [ ]:
param_x = np.array([[1, 2, 3, 4, 5, 6, 7, 8]])
param_y = np.array([[1]])

In [ ]:
model = ANNClassifier(param_x, param_y, learning_rate=0.1, verbose=100)
model.add(12, activation_function=sigmoid)
model.add(8, activation_function=sigmoid)
model.add(1, activation_function=sigmoid)

In [ ]:
model.summary()

SUMMARY
Layer (Type)	: Input Layer
Activation	: None
Output Shape	: 8
Param		: -
Layer (Type)	: dense_0 (Dense)
Activation	: sigmoid
Output Shape	: 12
Param		: 108
Weight		:
	node_0 [0. 0. 0. 0. 0. 0. 0. 0.]
	node_1 [0. 0. 0. 0. 0. 0. 0. 0.]
	node_2 [0. 0. 0. 0. 0. 0. 0. 0.]
	node_3 [0. 0. 0. 0. 0. 0. 0. 0.]
	node_4 [0. 0. 0. 0. 0. 0. 0. 0.]
	node_5 [0. 0. 0. 0. 0. 0. 0. 0.]
	node_6 [0. 0. 0. 0. 0. 0. 0. 0.]
	node_7 [0. 0. 0. 0. 0. 0. 0. 0.]
	node_8 [0. 0. 0. 0. 0. 0. 0. 0.]
	node_9 [0. 0. 0. 0. 0. 0. 0. 0.]
	node_10 [0. 0. 0. 0. 0. 0. 0. 0.]
	node_11 [0. 0. 0. 0. 0. 0. 0. 0.]
Layer (Type)	: dense_1 (Dense)
Activation	: sigmoid
Output Shape	: 8
Param		: 104
Weight		:
	node_0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
	node_1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
	node_2 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
	node_3 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
	node_4 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
	node_5 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
	node_6 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
	n

## Export / Import

In [ ]:
# save model
ann.save('model.json')

Export:
[{'activation': 'sigmoid',
  'bias': array([-1.8668683 ,  0.22345415, -0.664927  , -1.2359833 , -0.88661206,
       -0.6275985 , -0.7017379 , -0.6822324 ], dtype=float32),
  'nodes': 8,
  'weights': array([[ 3.2450957 , -4.2258496 ],
       [-2.5172458 , -2.9276252 ],
       [-0.03413637, -0.6484623 ],
       [-3.300996  ,  2.278239  ],
       [-0.3874242 ,  0.06884778],
       [-1.6303849 ,  0.60353094],
       [ 0.4656466 , -0.08640727],
       [-0.645144  , -0.4315381 ]], dtype=float32)},
 {'activation': 'softmax',
  'bias': array([ 0.9617703, -0.9617696], dtype=float32),
  'nodes': 2,
  'weights': array([[-2.7685611 ,  2.6373317 ,  0.31633526, -2.588205  ,  0.5865817 ,
        -0.36277023,  0.21714655, -0.06120121],
       [ 3.1873586 , -1.6696097 , -0.08435065,  2.2522638 ,  0.69639504,
         1.2915646 , -0.78183573, -0.21638407]], dtype=float32)}]


In [ ]:
new_ann = ANNClassifier(x, y, learning_rate=0.2, verbose=2)
new_ann.load('model.json')

Import : 
[{'activation': 'sigmoid',
  'bias': [-1.866868257522583,
           0.22345414757728577,
           -0.6649270057678223,
           -1.2359832525253296,
           -0.886612057685852,
           -0.6275985240936279,
           -0.7017378807067871,
           -0.6822323799133301],
  'nodes': 8,
  'weights': [[3.245095729827881, -4.225849628448486],
              [-2.5172457695007324, -2.9276251792907715],
              [-0.034136366099119186, -0.6484622955322266],
              [-3.3009960651397705, 2.2782390117645264],
              [-0.38742420077323914, 0.06884778290987015],
              [-1.630384922027588, 0.6035309433937073],
              [0.4656465947628021, -0.08640727400779724],
              [-0.645143985748291, -0.43153810501098633]]},
 {'activation': 'softmax',
  'bias': [0.9617702960968018, -0.9617695808410645],
  'nodes': 2,
  'weights': [[-2.7685611248016357,
               2.63733172416687,
               0.3163352608680725,
               -2.588205099105835

In [ ]:
new_predict = new_ann.predict(x)
new_predict

array([[0.91219728, 0.08780272],
       [0.14021183, 0.85978817],
       [0.11261296, 0.88738704],
       [0.80164388, 0.19835612]])

In [ ]:
new_ann.summary()

SUMMARY
Layer (Type)	: Input Layer
Activation	: None
Output Shape	: 2
Param		: -
Layer (Type)	: dense_0 (Dense)
Activation	: sigmoid
Output Shape	: 8
Param		: 24
Weight		:
	node_0 [ 3.24509573 -4.22584963]
	node_1 [-2.51724577 -2.92762518]
	node_2 [-0.03413637 -0.6484623 ]
	node_3 [-3.30099607  2.27823901]
	node_4 [-0.3874242   0.06884778]
	node_5 [-1.63038492  0.60353094]
	node_6 [ 0.46564659 -0.08640727]
	node_7 [-0.64514399 -0.43153811]
Layer (Type)	: dense_1 (Dense)
Activation	: softmax
Output Shape	: 2
Param		: 18
Weight		:
	node_0 [-2.76856112  2.63733172  0.31633526 -2.5882051   0.58658171 -0.36277023
  0.21714655 -0.06120121]
	node_1 [ 3.18735862 -1.66960967 -0.08435065  2.25226378  0.69639504  1.29156458
 -0.78183573 -0.21638407]
Total params : 42
